# Etapa 1 - Reto

Integrantes de equipo:

Rodrigo Galván Paiz | A01721158

Andrés Aguirre Rodríguez | A01284373

Andres Fernando Garza Garcia | A01138704

Daira Adriana Chavarria Rodriguez | A01274745


----------------------------------------------------------

En esta entrega se enseña una simulacion simple de como es que funciona una interseccion con dos semaforos. El proposito de esto es tener los fundamentos de lo que va a ser la simulacion final. Ademas en esta entrega se describe a fondo con comentarios las condicionales y el codigo en general que hace que esta simulacion funcione.

In [ ]:
pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 9.3 MB/s 
     |████████████████████████████████| 66 kB 3.3 MB/s 


In [ ]:
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid
from mesa.space import MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
#%matplotlib inline
import matplotlib
import itertools
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd
import json

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

In [ ]:

class Car(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    #0 Derecha y 1 Abajo
    self.direction = 0
    self.wait = False
    self.time = 2
    self.waitingInRedLight = False
    # Por defecto, los coches se están moviendo (true)
    self.is_moving = True
    self.randD = random.randint(1,2)
    #Color del carro
    color = random.randint(4,6)
    if color == 4:
      self.color = 'white'
    elif color == 5:
      self.color = 'green2'
    else:
      self.color = 'other'


  def step(self):
    
    self.randD = random.randint(1,6)

    if(self.pos[0] == self.model.grid.width // 2 and self.pos[1] == self.model.grid.height // 2):
      if(self.randD == 1):
        self.direction = 3
      elif(self.randD == 6):
        self.direction = 1
    if(self.pos[0] == self.model.grid.width // 2 + 1 and self.pos[1] == self.model.grid.height // 2):
      if(self.randD == 1):
        self.direction = 3
      elif(self.randD == 6):
        self.direction = 0
    if(self.pos[0] == self.model.grid.width // 2 + 1 and self.pos[1] == self.model.grid.height // 2 + 1):
      if(self.randD == 1):
        self.direction = 2
      elif(self.randD == 6):
        self.direction = 0
    if(self.pos[0] == self.model.grid.width // 2 and self.pos[1] == self.model.grid.height // 2 + 1):
      if(self.randD == 1):
        self.direction = 2
      elif(self.randD == 6):
        self.direction = 1
    # Obtenemos todos los vecinos
    neighbours = self.model.grid.get_neighbors(
      self.pos,
      moore = False, # No se trabaja con diagonales
      include_center = False) # No se incluye al centro (self)

    #La luz esta en verde por default hasta que detectemos semaforo rojo
    red_light = False
    
    for neighbour in neighbours:
      # Si está el semáforo de vecino y está en ROJO, no podemos avanzar
      if isinstance(neighbour, Traffic_Light) and neighbour.color == 'red': 
        red_light = True
        self.is_moving = False
        self.waitingInRedLight = True
      if isinstance(neighbour, Traffic_Light) and neighbour.color == 'green' and self.waitingInRedLight == True:
        self.wait = True
    
    # Si no está en rojo y se puede avanzar, se mueve el coche
    if not red_light and self.CanMove():
      if(self.wait == True):
        if(self.time == 0):
          self.time = 2
          self.Move()
          self.wait = False
          self.waitingInRedLight = False
        else:
          self.time -= 1
      else:
        self.Move()

    

  def Move(self):

    # 0 = Derecha
    # 1 = Izquierda
    # 2 = Arriba
    # 3 = Abajo

    self.is_moving = True
    # Mover el coche hacia la derecha o abajo
    #dependiendo de como se instancio
    if self.direction == 0:
      self.moveRight()
    if self.direction == 1:
      self.moveLeft()
    if self.direction == 2:
      self.moveUp()
    if self.direction == 3:
      self.moveDown()
		
  def moveDown(self):
    # Cambiar la posicion en "y" del carro
    newPos = (self.pos[0] + 1, self.pos[1])
    if(self.pos[0] == self.model.grid.height - 1):
      if(self.model.grid.is_cell_empty([0, self.pos[1]])):
        self.model.grid.move_agent(self, newPos)
    else:
      self.model.grid.move_agent(self, newPos)
  
  def moveUp(self):
    # Cambiar la posicion en "y" del carro
    newPos = (self.pos[0] - 1, self.pos[1])
    if(self.pos[0] == 0):
      if(self.model.grid.is_cell_empty( [self.model.grid.height - 1, self.pos[1]])):
        self.model.grid.move_agent(self, newPos)
    else:
      self.model.grid.move_agent(self, newPos)

  def moveRight(self):
    # Cambiar la posicion de "x" del carro
    newPos = (self.pos[0], self.pos[1] + 1)
    if(self.pos[1] ==  self.model.grid.width - 1):
      if(self.model.grid.is_cell_empty( [self.pos[0], 0])):
        self.model.grid.move_agent(self, newPos)
    else:
      self.model.grid.move_agent(self, newPos)

  def moveLeft(self):
    # Cambiar la posicion de "x" del carro
    newPos = (self.pos[0], self.pos[1] - 1)
    if(self.pos[1] ==  0):
      if(self.model.grid.is_cell_empty( [self.pos[0], self.model.grid.width - 1])):
        self.model.grid.move_agent(self, newPos)
    else:
      self.model.grid.move_agent(self, newPos)
	
	# Para ver si el SIGUIENTE coche en la MISMA dirección está detenido o no
  def CanMove(self):
  
    neighbours = self.model.grid.get_neighbors(
        self.pos,
        moore = False, # No se trabaja con diagonales
        include_center = False) # No se incluye al centro (self)
    
    # 0 = Derecha
    # 1 = Izquierda
    # 2 = Arriba
    # 3 = Abajo

    for neighbour in neighbours:
      # Si lo que hay delante es un coche
      if isinstance(neighbour, Car):
        if(self.direction == 0 and neighbour.pos[1] == self.pos[1] + 1):
          # True si se está moviendo, False de estar detenido
          return False
        elif (self.direction == 1 and neighbour.pos[1] == self.pos[1] - 1):
          return False
        elif (self.direction == 2 and neighbour.pos[0] == self.pos[0] - 1):
          return False
        elif (self.direction == 3 and neighbour.pos[0] == self.pos[0] + 1):
          return False
  
    # De lo contrario regresa True dado que no hay nada estorbando (no hay coche)
    else: return True
  
  
  
class Traffic_Light(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    #Semaforo en rojo
    self.color = 'red'
    #Limite de tiempo en el que puede estar en rojo
    self.time = 15
    #Si paso limite de tiempo esta se vuelve en true
    self.stopByTimer = False
    
  #Funcion para cambiar semaforo a verde
  def toGreen(self):
    self.color = 'green'
    
  #Funcion para cambiar semaforo a rojo
  def toRed(self):
    self.color = 'red'

  def otherTrafficLight(self):
		# Encontrar el otro semáforo en el modelo
    tl_list = self.model.getTrafficLights()
    for tl in tl_list:
      if tl.unique_id[0] != self.pos[0] and tl.unique_id[1] != self.pos[1]:
        if tl.color == 'green':
          return False
    return True
	
  def step(self):
    #Si el semaforo esta en verde se le resta al cotador
    if self.color == 'green':
      self.time -= 1
    elif self.color == 'red':
      #Si el semaforo esta en rojo se resetea el contador
      self.time = 15
      self.stopByTimer = False
    
    #Si paso el tiempo limite se pone en rono y es true la variable indicada
    if self.time == 0:
      self.toRed()
      self.stopByTimer = True

    #Booleana de si hay carro al lado del semaforo o no
    any_car = False
    
    neighbours = self.model.grid.get_neighbors(
			self.pos,
			moore = True, 
			include_center = False) # No se incluye al centro (self)
    
    for neighbour in neighbours:
      if isinstance(neighbour, Car):
        any_car = True # Si hay coches esperando cruzar la calle
      
    #Si el semaforo esta en rojo, hay carros y no se acaba de 
    #pasar el limite de tiempo vuelve en verde la luz
    if self.otherTrafficLight() == True and any_car and not self.stopByTimer:
      self.toGreen()

    #Si no hay carros entonces es rojo
    if not any_car:
      self.toRed()
    

      

class Grass(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.color = 'olive'		


In [ ]:
jsons = []
# Clase que define El grid y las reglas del juego
class CrossRoad(Model):
    def __init__(self, w, h, cs):
      self.num_agents = w * h # Tamaño del entorno
      self.total = 0
      self.carsNeeded = cs
      self.width = w - 1
      self.height = h - 1
      
      self.carCoordinate_toRight = (self.width // 2 + 1, 0)
      self.carCoordinate_toLeft = (self.width // 2, self.height)
      self.carCoordinate_toUp = (self.width, self.height // 2 + 1)
      self.carCoordinate_toDown = (0, self.height // 2)

      # 4 semáforos para el modelo escogido (actualizado)
      self.traffic_lights = [] #Lista de semaforos

      # Single Grid porque no hay superposición
      self.grid = SingleGrid(w, h, True)
      self.schedule = SimultaneousActivation(self)
      self.time = 0 #Tiempo entre que se instancian carros

      pos_x = self.width//2		# Mitad de la matriz, para saber dónde ubicar las calles y semáforos
      pos_y = self.height//2	
      
      self.agents_list = []

      # Encontramos la posición de los semáforos
      #BottomLeft
      tl_x1 = pos_x + 2
      tl_y1 = pos_y - 2

      #TopRight
      tl_x2 = pos_x - 1
      tl_y2 = pos_y + 3

      tl_x3 = pos_x - 2
      tl_y3 = pos_y - 1

      tl_x4 = pos_x + 3
      tl_y4 = pos_x + 2


      # Creamos el pasto (grass agent)
      for (content, x, y) in self.grid.coord_iter():
        # Hacemos las calles de grosor = 3 para no tener problemas con semáforos
          
        if (x != pos_x and x != pos_x + 1) and (y != pos_y + 1 and y != pos_y):

            # Posicionamos los semáforos
            if (x == tl_x1 and y == tl_y1) or (x == tl_x2 and y == tl_y2) or (x == tl_x3 and y == tl_y3) or (x == tl_x4 and y == tl_y4):
              a = Traffic_Light((x, y), self)
              self.traffic_lights.append(a)
              self.grid.place_agent(a, (x, y))
              self.schedule.add(a)

            #Posicionamos pasto
            else:
              g = Grass((x,y), self)
              self.grid.place_agent(g, (x, y))
	
      # Aquí definimos el colector de datos para obtener el grid completo.
      self.datacollector = DataCollector(
        model_reporters = { "Grid" : self.get_grid}
      )

    #Obtener semaforos
    def getTrafficLights(self):
      return self.traffic_lights

    def canGenerateCar(self, position):
      if position == self.carCoordinate_toRight:
        cell_obj = self.grid[self.width // 2 + 1][self.width]
      if position == self.carCoordinate_toLeft:
        cell_obj = self.grid[self.width // 2][0]
      if position == self.carCoordinate_toUp:
        cell_obj = self.grid[0][self.height // 2 + 1]
      if position == self.carCoordinate_toDown:
        cell_obj = self.grid[0][self.height // 2]

      if isinstance(cell_obj, Car):
        if cell_obj.is_moving:
          return False

      return True

    def modify(self, newCar):
      self.schedule.add(newCar)
      self.total += 1
      self.time = 2
      self.carsNeeded -= 1

    def step(self):

      #Cada step instanciamos carros
      
      # 0 = Derecha
      # 1 = Izquierda
      # 2 = Arriba
      # 3 = Abajo

      
      #Si el total de carros es menor a n y el timer es 0
      if(self.total <= self.carsNeeded and self.time == 0):

        car_dir = random.randint(0,3) # Escogemos una dirección aleatoria para nuestros coches
        print ("Car Direction: " + str(car_dir))

        #Si total es par carro se mueve a derecha
        if car_dir == 0:
          #Si no hay nada en la celda donde se va a poner el carro
          if(self.grid.is_cell_empty((self.carCoordinate_toRight)) == True) and (self.canGenerateCar(self.carCoordinate_toRight)):
            #Instanciar carro, sumar al total y resetear el timer
            c = Car((self.total), self)
            self.grid.place_agent(c, (self.carCoordinate_toRight))
            self.modify(c)

        if car_dir == 1:
          #Si no hay nada en la celda donde se va a poner el carro
          if(self.grid.is_cell_empty((self.carCoordinate_toLeft)) == True) and (self.canGenerateCar(self.carCoordinate_toLeft)):
            #Instanciar carro, sumar al total y resetear el timer
            c = Car((self.total), self)
            c.direction = 1
            self.grid.place_agent(c, (self.carCoordinate_toLeft))
            self.modify(c)

        if car_dir == 2:
          #Si no hay nada en la celda donde se va a poner el carro
          if(self.grid.is_cell_empty((self.carCoordinate_toUp)) == True) and (self.canGenerateCar(self.carCoordinate_toUp)):
            #Instanciar carro, sumar al total y resetear el timer
            c = Car((self.total), self)
            c.direction = 2
            self.grid.place_agent(c, (self.carCoordinate_toUp))
            self.modify(c)

        if car_dir == 3:
          #Si no hay nada en la celda donde se va a poner el carro
          if(self.grid.is_cell_empty((self.carCoordinate_toDown)) == True) and (self.canGenerateCar(self.carCoordinate_toDown)):
            #Instanciar carro, sumar al total y resetear el timer
            c = Car((self.total), self)
            c.direction = 3
            self.grid.place_agent(c, (self.carCoordinate_toDown))
            self.modify(c)

      #Se resta al timer que controla la frequencia en la que se instancian
      #los carros
      self.time -= 1
      jsons.append(self.read_agents())
      self.datacollector.collect(self)
      self.schedule.step()  

    #Comunicacion JSON. En cada step del modelo se llama esta funcion y se generan los jsons
    def read_agents(self):
      self.agents_list = []
      #Por todas las celdas del grid
      for cell in self.grid.coord_iter():
          agent, x, y = cell

          if isinstance(agent, Car):
              agent_dict = {
                  'id': str(agent.unique_id),
                  'kind': 'car',
                  'positionX': str(agent.pos[0]),
                  'positionZ': str(agent.pos[1]),
                  'color': str(agent.color)
              }
              self.agents_list.append(agent_dict)

          elif isinstance(agent, Traffic_Light  ):
              agent_dict = {
                  'id': str(agent.unique_id),
                  'kind': 'traffictLight',
                  'positionX': str(agent.pos[0]),
                  'positionZ': str(agent.pos[1]),
                  'color': str(agent.color)
              }
              self.agents_list.append(agent_dict)
          
          

      return self.agents_list

    def get_grid(model):
      #Definir los colores en el grid
      grid = np.zeros((model.grid.width, model.grid.height))

      #Por todas las celdas del grid
      for cell in model.grid.coord_iter():
        agent, x, y = cell		

        if isinstance(agent, Car):
            if agent.color == 'white':
                grid[x][y] = 4
            elif agent.color == 'green2':
              grid[x][y] = 5
            else:
              grid[x][y] = 6

        elif isinstance(agent, Grass):
            if agent.color == 'olive':
                grid[x][y] = 3

        elif isinstance(agent, Traffic_Light):
            if agent.color == 'green':
                grid[x][y] = 2
            else: # red
                grid[x][y] = 1

        else: # Street
            grid[x][y] = 0
      return grid

# Definimos el tamaño del Grid
GRID_SIZE = 21

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 100

NUM_CARS = 100

# Corremos el modelo
model = CrossRoad(GRID_SIZE, GRID_SIZE, NUM_CARS)
for i in range(NUM_GENERATIONS):
    model.step()

with open("json_file5.txt", "w") as archivo:
    json.dump(jsons, archivo)

Car Direction: 2
Car Direction: 3
Car Direction: 3
Car Direction: 2
Car Direction: 2
Car Direction: 1
Car Direction: 2
Car Direction: 2
Car Direction: 2
Car Direction: 1
Car Direction: 0
Car Direction: 2
Car Direction: 2
Car Direction: 3
Car Direction: 3
Car Direction: 1
Car Direction: 0


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])

#Los colores utilizados
cmap = matplotlib.cm.get_cmap('viridis', 7) # Puede ser cualquier otra
cmap = cmap(np.linspace(0, 1, 7))
cmap[0] = np.array([50/256, 50/256, 50/256, 1])   # Calle en gris
cmap[1] = np.array([256/256, 0/256, 0/256, 1])    # Semáforo en rojo
cmap[2] = np.array([0/256, 256/256, 0/256, 1])    # Semáforo en verde
cmap[3] = np.array([96/256, 170/256, 70/256, 1])  # Pasto verde  
cmap[4] = np.array([0/256, 50/256, 100/256, 1])   # Carro azul 
cmap[5] = np.array([100/256, 50/256, 100/256, 1])   # Carro azul 
cmap[6] = np.array([150/256, 100/256, 100/256, 1])   # Carro azul


new_cmap = matplotlib.colors.ListedColormap(cmap)

patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)



def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim